# Phonemes (tłumaczenie)

[Źródło](https://arxiv.org/pdf/1409.0473.pdf)

Wiedząc już jak działa sieć kodująca/dekodująca i jak działają sieci rekurencyjne możemy spróbować zbudować sieć zdolną do tłumaczenia tekstu.

Intuicyjnie tym co chcemy zrobić jest stworzenie sieci kodującej/dekodującej, która będzie wstanie zakodować informację napisaną w jednym języku w formie wektora zmiennych ukrytych i następnie odkodować ją w innym języku.

De facto polega to na jednoczesnym trenowaniu dwóch (lub więcej) sieci - każda sieć jest autoencoderem dla każdego z rozpatrywanych języków. Trenuje się je jednocześnie aby uzyskać takie same lub zbliżone wartości wektorów zmiennych ukrytych. Tłumaczenie polega po prostu na "mieszaniu" ze sobą otrzymanych modeli.

Takie podejście ma jedną zasadniczą wadę - otrzymany model może sobie nie radzić z tłumaczeniem długich i wyraźnie odbiegających od zbioru trenującego tekstów. Dlatego konieczne jest zaproponowanie innego podejścia.

W tym wypadku zbudujemy model, który będzie kodował tekst nie w pojedynczy wektor ale w ich cały zbiór (którego każdy element będzie odpowiadał pojedynczemu słowu, bądź ich zbitce), a następnie odpowiednio losował i rozmieszczał elementy tego zbioru w dekodowanym tekście.

Najpierw wczytajmy dane:

In [1]:
using Flux, Flux.Data.CMUDict
using Flux: onehot, batchseq
using Base.Iterators: partition

dict = cmudict()
alphabet = [:end, CMUDict.alphabet()...]
phones = [:start, :end, CMUDict.symbols()...]

tokenise(s, α) = [onehot(c, α) for c in s]

# Turn a word into a sequence of vectors
tokenise("PHYLOGENY", alphabet)
# Same for phoneme lists
tokenise(dict["PHYLOGENY"], phones)

words = sort(collect(keys(dict)), by = length)

# Finally, create iterators for our inputs and outputs.
batches(xs, p) = [batchseq(b, p) for b in partition(xs, 50)]

Xs = batches([tokenise(word, alphabet) for word in words],
             onehot(:end, alphabet))

Ys = batches([tokenise([dict[word]..., :end], phones) for word in words],
             onehot(:end, phones))

Yo = batches([tokenise([:start, dict[word]...], phones) for word in words],
             onehot(:end, phones))

data = collect(zip(Xs, Yo, Ys))

┌ Info: Downloading CMUDict dataset
└ @ Flux.Data.CMUDict C:\Users\p\.julia\packages\Flux\zNlBL\src\data\cmudict.jl:19


2352-element Array{Tuple{Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1},Array{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},1}},1}:
 ([[false false … false false; false false … false false; … ; false false … false false; false false … false false], [true true … false false; false false … false false; … ; false false … false false; false false … true false]], [[true true … true true; false false … false false; … ; false false … false false; false false … false false], [false false … false false; false false … false false; … ; false false … true false; false false … false false], [false false … false false; false false … false false; … ; false false … false false; false false … false false], [false false … false false; true true … true false; … ; false false … false false; false false … false false], [false false … false false; true true … true false; … ; false false … false false; false false … false false], [false false … f

I przejdźmy do budowy sieci:

In [2]:
using Flux: flip, crossentropy, reset!, throttle

Zacznijmy od opisu i budowy sieci kodującej tekst. Jest ona dwukierunkowa, co jest szczególnie przydatne w przypadku języków nie mających ścisłego szyku wyrazów i w których słowo tworzy wiele morfemów jednocześnie:

In [3]:
Nin = length(alphabet)
Nh = 30 # size of hidden layer

# A recurrent model which takes a token and returns a context-dependent
# annotation.

forward  = LSTM(Nin, Nh÷2)
backward = LSTM(Nin, Nh÷2)
encode(tokens) = vcat.(forward.(tokens), flip(backward, tokens))

alignnet = Dense(2Nh, 1)
align(s, t) = alignnet(vcat(t, s .* trues(1, size(t, 2))))


align (generic function with 1 method)

Sieć dekodująca posiada ciekawszą budowę. jej podstawowym elementem jest wektor <i>adnotacji</i> $(h_1,h_2,\dots,h_T)$, które są zmiennymi ukrytymi wygenerowanymi przez sieć kodującą.
Te zmienne są przetwarzane do  postaci:

 $c_i = \sum_{i=1}^T\alpha_{ij}h_j$
 
 
 $\alpha_{ij} = \frac{exp(e_{ij})}{\sum_{k=1}^Te_{ik}}$ jest wagą każdej adnotacji a $e_{ij}$ jest modelem rozmieszczenia i przyjmuje postać:
 
 $e_{ij} = a(s_{j-1}h_j)$
 
 gdzie $s_{j-1}$ jest <i> ukrytym stanem </i> modelu.

In [4]:

# A recurrent model which takes a sequence of annotations, attends, and returns
# a predicted output token.

recur   = LSTM(Nh+length(phones), Nh)
toalpha = Dense(Nh, length(phones))

function asoftmax(xs)
  xs = [exp.(x) for x in xs]
  s = sum(xs)
  return [x ./ s for x in xs]
end

function decode1(tokens, phone)
  weights = asoftmax([align(recur.state[2], t) for t in tokens])
  context = sum(map((a, b) -> a .* b, weights, tokens))
  y = recur(vcat(Float32.(phone), context))
  return softmax(toalpha(y))
end

decode(tokens, phones) = [decode1(tokens, phone) for phone in phones]

# The full model

state = (forward, backward, alignnet, recur, toalpha)

function model(x, y)
  ŷ = decode(encode(x), y)
  reset!(state)
  return ŷ
end

model (generic function with 1 method)

Możemy zacząć uczyć sieć:

In [5]:
loss(x, yo, y) = sum(crossentropy.(model(x, yo), y))

evalcb = () -> @show loss(data[500]...)
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [6]:
Flux.train!(loss, params(state), data, opt, cb = throttle(evalcb, 10))

loss(data[500]...) = 30.970905f0 (tracked)
loss(data[500]...) = 19.860256f0 (tracked)
loss(data[500]...) = 18.348082f0 (tracked)
loss(data[500]...) = 16.987936f0 (tracked)
loss(data[500]...) = 15.86444f0 (tracked)
loss(data[500]...) = 15.002179f0 (tracked)
loss(data[500]...) = 14.764329f0 (tracked)
loss(data[500]...) = 14.152394f0 (tracked)
loss(data[500]...) = 13.6408825f0 (tracked)
loss(data[500]...) = 13.180557f0 (tracked)
loss(data[500]...) = 14.0899105f0 (tracked)
loss(data[500]...) = 13.9690485f0 (tracked)
loss(data[500]...) = 13.257049f0 (tracked)
loss(data[500]...) = 13.371975f0 (tracked)
loss(data[500]...) = 14.476399f0 (tracked)
loss(data[500]...) = 14.182317f0 (tracked)
loss(data[500]...) = 13.972023f0 (tracked)
loss(data[500]...) = 15.613295f0 (tracked)
loss(data[500]...) = 15.468822f0 (tracked)
loss(data[500]...) = 16.686935f0 (tracked)
loss(data[500]...) = 17.864323f0 (tracked)
loss(data[500]...) = 19.41148f0 (tracked)


Na wszelki wypadek zapiszmy otrzymane wyniki:

In [7]:
using BSON: @save, @load

┌ Info: Recompiling stale cache file C:\Users\p\.julia\compiled\v1.0\BSON\3tVCZ.ji for BSON [fbb218c0-5317-5bc6-957e-2ee96dd4b1f0]
└ @ Base loading.jl:1184


In [ ]:
weights = Tracker.data.(params(state));

In [ ]:
@save "phonemes.bson" weights

In [ ]:
@load "phonemes.bson" weights

In [ ]:
Flux.loadparams!(state, weights)

In [ ]:
using StatsBase: wsample

function predict(s)
  ts = encode(tokenise(s, alphabet))
  ps = Any[:start]
  for i = 1:50
    dist = decode1(ts, onehot(ps[end], phones))
    next = wsample(phones, vec(Tracker.data(dist)))
    next == :end && break
    push!(ps, next)
  end
  return ps[2:end]
end

predict("PHYLOGENY")

In [ ]:
predict("PHYLOGENY")
